# Projet IA - GODELLE Damien

Après plusieurs tentatives de faire la récupération des images et profils des caractères via les API en local, j'ai finalement basculé sur un environnement virtuel dont ce notebook reprend le contenu. J'avais également tenté de récupérer l'audio ou de récupérer plusieurs images mais cela ne fonctionnait pas correctement voir pas du tout en local donc j'ai abandonné cette partie et ai essayé d'avoir une API Wikipédia et AniList fonctionnel.

Actuellement j'ai donc:
- Utilisé l'API Wikipédia pour récupérer la description des pages que l'on souhaite puis la création d'un résumé de ces pages pour que cela soit plus pratique pour la suite.
- Utilisé l'API de Anilist pour récupérer la description d'un caractère ou d'un manga/animé 

Les 2 codes permettent de télécharger des fichiers en .json et .html avec les descriptions souhaitées.

Après plusieurs tentatives de faire la récupération des images et profils des caractères via les API en local, j'ai finalement basculé sur un environnement virtuel dont ce notebook reprend le contenu. J'avais également tenté de récupérer l'audio ou de récupérer plusieurs images mais cela ne fonctionnait pas correctement voir pas du tout en local donc j'ai abandonné cette partie et ai essayé d'avoir une API Wikipédia et AniList fonctionnel.

Actuellement j'ai donc:
- Utilisé l'API Wikipédia pour récupérer la description des pages que l'on souhaite puis la création d'un résumé de ces pages pour que cela soit plus pratique pour la suite.
- Utilisé l'API de Anilist pour récupérer la description d'un caractère ou d'un manga/animé 

Les 2 codes permettent de télécharger des fichiers en .json et .html avec les descriptions souhaitées.

## Préliminaire: Installation

In [2]:
%pip install wikipedia
%pip install wikipedia-api
%pip install pyannote.audio
%pip install pydub

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Partie récupération avec API Wikipéddia

In [5]:
import wikipedia
import wikipediaapi
import json
import os

class WikipediaFetcher:
    """
    Classe pour récupérer les données d'une page Wikipédia avec des fonctionnalités avancées.
    """

    def __init__(self, language="en", user_agent="DamienGodelleProject/1.0 (damien.godelle@gmail.com)"):
        self.language = language
        self.user_agent = user_agent
        self.wiki_client = wikipediaapi.Wikipedia(language=language, user_agent=user_agent)
        wikipedia.set_lang(language)

    def extract_sections(self, sections):
        """
        Parcourt récursivement les sections et sous-sections d'une page Wikipédia.
        """
        return [
            {
                "titre": section.title,
                "contenu": section.text[:],
                "sous_sections": self.extract_sections(section.sections),
            }
            for section in sections
        ]

    def search_titles(self, query, max_results=5):
        """
        Recherche les titres de pages correspondantes à une requête donnée.
        """
        results = wikipedia.search(query, results=max_results, suggestion=True)
        if not results or not results[0]:
            raise ValueError("Aucun résultat trouvé pour cette recherche.")
        return results[0]  # Renvoie les propositions

    def fetch_page(self, title):
        """
        Récupère une page Wikipédia par son titre.
        """
        page = self.wiki_client.page(title)
        if not page.exists():
            raise ValueError(f"La page '{title}' n'existe pas.")
        return page

    def get_page_data(self, query, summary_only=False, save_path=None, save_as_html=False):
        """
        Récupère les données d'une page Wikipédia avec ses sections.
        """
        title = self.search_titles(query)[0]
        page = self.fetch_page(title)
        data = {
            "titre": page.title,
            "resume": page.summary,
        }

        if not summary_only:
            data["sections"] = self.extract_sections(page.sections)

        if save_path:
            if save_as_html:
                self.save_to_html(data, save_path)
            else:
                self.save_to_json(data, save_path)

        return data

    @staticmethod
    def save_to_json(data, path):
        """
        Enregistre les données dans un fichier JSON.
        """
        with open(path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
            print(f"Données enregistrées dans : {os.path.abspath(path)}")

    @staticmethod
    def save_to_html(data, path):
        """
        Enregistre les données dans un fichier HTML.
        """
        html_content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>{data['titre']}</title>
        </head>
        <body>
            <h1>{data['titre']}</h1>
            <p><strong>Résumé :</strong> {data['resume']}</p>
        """

        if "sections" in data:
            html_content += "<h2>Sections</h2><ul>"
            for section in data["sections"]:
                html_content += WikipediaFetcher.format_section_to_html(section)
            html_content += "</ul>"

        html_content += """
        </body>
        </html>
        """

        with open(path, 'w', encoding='utf-8') as file:
            file.write(html_content)
            print(f"Données enregistrées en HTML dans : {os.path.abspath(path)}")

    @staticmethod
    def format_section_to_html(section):
        """
        Convertit une section et ses sous-sections en HTML.
        """
        html = f"<li><h3>{section['titre']}</h3><p>{section['contenu']}</p>"
        if "sous_sections" in section and section["sous_sections"]:
            html += "<ul>"
            for subsection in section["sous_sections"]:
                html += WikipediaFetcher.format_section_to_html(subsection)
            html += "</ul>"
        html += "</li>"
        return html

# Exemple d'utilisation
if __name__ == "__main__":
    fetcher = WikipediaFetcher(language="fr")
    data = fetcher.get_page_data("Ramen", summary_only=False, save_path="wikipedia_data.html", save_as_html=True)
    print(data)


Données enregistrées en HTML dans : C:\Users\mbiver\Desktop\IA\Projet\wikipedia_data.html
{'titre': 'Rāmen', 'resume': "Un rāmen (ラーメン, prononcé en français [ʀamɛn] ou [lamɛn]) est une recette de cuisine japonaise héritée de la cuisine chinoise et adaptée au goût des habitants de l'archipel nippon. Le mot rāmen est emprunté par le japonais de son nom original chinois en mandarin [lāmiàn] (拉麵, 拉面 'nouilles tirées'). Dans sa forme traditionnelle, il s'agit d'une soupe de nouilles, à base de bouillon agrémenté de nombreuses variantes d'ingrédients animaux, végétaux et aromates (poissons, viandes, légumes, algues, œuf, etc.). Importé de Chine à la fin du XIXe siècle, il est à ce jour considéré comme faisant partie des plats emblématiques de la gastronomie japonaise.", 'sections': [{'titre': 'Historique', 'contenu': "Plat dont les premières versions étaient d'origine chinoise, le rāmen (ラーメン／拉麺／老麺／柳麺) tirerait son nom actuel des lamians (拉面 / 拉麺, lā miàn, « nouilles tirées »), des pâtes de 

### Résumé de la page wiki récupéré

In [6]:
if __name__ == "__main__":
    # Crée une instance de WikipediaFetcher
    fetcher = WikipediaFetcher(language="fr")
    
    # Requête pour obtenir uniquement le résumé
    try:
        data = fetcher.get_page_data(
            query="Ramen",  # Le sujet de la recherche
            summary_only=True  # Indique qu'on ne veut que le résumé
        )
        print("Résumé :")
        print(data["resume"])  # Affiche le résumé
    except ValueError as e:
        print(f"Erreur : {e}")


Résumé :
Un rāmen (ラーメン, prononcé en français [ʀamɛn] ou [lamɛn]) est une recette de cuisine japonaise héritée de la cuisine chinoise et adaptée au goût des habitants de l'archipel nippon. Le mot rāmen est emprunté par le japonais de son nom original chinois en mandarin [lāmiàn] (拉麵, 拉面 'nouilles tirées'). Dans sa forme traditionnelle, il s'agit d'une soupe de nouilles, à base de bouillon agrémenté de nombreuses variantes d'ingrédients animaux, végétaux et aromates (poissons, viandes, légumes, algues, œuf, etc.). Importé de Chine à la fin du XIXe siècle, il est à ce jour considéré comme faisant partie des plats emblématiques de la gastronomie japonaise.


## Partie récupération avec API AniList

In [8]:
import requests
import json
import os

class AniListFetcher:
    """
    Classe pour interagir avec l'API AniList et récupérer des données sur les animes, mangas, ou personnages.
    """

    API_URL = "https://graphql.anilist.co"

    def __init__(self):
        pass

    def search_titles(self, query, media_type="ANIME", max_results=5):
        """
        Recherche des titres sur AniList par type de média (ANIME, MANGA, CHARACTER).
        """
        query_body = {
            "query": """
                query ($search: String, $type: MediaType, $page: Int, $perPage: Int) {
                    Page(page: $page, perPage: $perPage) {
                        media(search: $search, type: $type) {
                            id
                            title {
                                romaji
                                english
                                native
                            }
                        }
                    }
                }
            """,
            "variables": {
                "search": query,
                "type": media_type,
                "page": 1,
                "perPage": max_results
            }
        }

        response = requests.post(self.API_URL, json=query_body)
        data = response.json()

        if "errors" in data:
            raise ValueError(f"Erreur lors de la recherche : {data['errors']}")

        results = [
            {
                "id": media["id"],
                "titles": media["title"]
            }
            for media in data["data"]["Page"]["media"]
        ]

        if not results:
            raise ValueError("Aucun résultat trouvé pour cette recherche.")
        return results

    def fetch_details(self, media_id, media_type="ANIME"):
        """
        Récupère les détails d'un média (anime ou manga) par son ID.
        """
        query_body = {
            "query": """
                query ($id: Int, $type: MediaType) {
                    Media(id: $id, type: $type) {
                        id
                        title {
                            romaji
                            english
                            native
                        }
                        description
                        genres
                        episodes
                        chapters
                        volumes
                        coverImage {
                            large
                        }
                        averageScore
                        popularity
                    }
                }
            """,
            "variables": {
                "id": media_id,
                "type": media_type
            }
        }

        response = requests.post(self.API_URL, json=query_body)
        data = response.json()

        if "errors" in data:
            raise ValueError(f"Erreur lors de la récupération des détails : {data['errors']}")

        return data["data"]["Media"]

    def save_to_json(self, data, path):
        """
        Enregistre les données dans un fichier JSON.
        """
        with open(path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
            print(f"Données enregistrées dans : {os.path.abspath(path)}")

    def save_to_html(self, data, path):
        """
        Enregistre les données dans un fichier HTML.
        """
        html_content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>{data['title']['romaji']}</title>
        </head>
        <body>
            <h1>{data['title']['romaji']} ({data['title']['english'] or 'N/A'})</h1>
            <img src="{data['coverImage']['large']}" alt="{data['title']['romaji']}">
            <p><strong>Description :</strong> {data['description']}</p>
            <p><strong>Genres :</strong> {', '.join(data['genres'])}</p>
            <p><strong>Score moyen :</strong> {data['averageScore']}</p>
            <p><strong>Popularité :</strong> {data['popularity']}</p>
            <p><strong>Épisodes :</strong> {data.get('episodes', 'N/A')}</p>
            <p><strong>Chapitres :</strong> {data.get('chapters', 'N/A')}</p>
            <p><strong>Volumes :</strong> {data.get('volumes', 'N/A')}</p>
        </body>
        </html>
        """

        with open(path, 'w', encoding='utf-8') as file:
            file.write(html_content)
            print(f"Données enregistrées en HTML dans : {os.path.abspath(path)}")


# Exemple d'utilisation
if __name__ == "__main__":
    fetcher = AniListFetcher()

    # Recherche de titres
    search_results = fetcher.search_titles("Attack on Titan", media_type="ANIME")
    print("Résultats de recherche :", search_results)

    # Récupération des détails
    media_id = search_results[0]["id"]
    details = fetcher.fetch_details(media_id, media_type="ANIME")
    print("Détails :", details)

    # Sauvegarde des données
    fetcher.save_to_json(details, "anilist_data.json")
    fetcher.save_to_html(details, "anilist_data.html")


Résultats de recherche : [{'id': 16498, 'titles': {'romaji': 'Shingeki no Kyojin', 'english': 'Attack on Titan', 'native': '進撃の巨人'}}, {'id': 110277, 'titles': {'romaji': 'Shingeki no Kyojin: The Final Season', 'english': 'Attack on Titan Final Season', 'native': '進撃の巨人 The Final Season'}}, {'id': 18397, 'titles': {'romaji': 'Shingeki no Kyojin OVA', 'english': 'Attack on Titan OVA', 'native': '進撃の巨人 OVA'}}, {'id': 20811, 'titles': {'romaji': 'Shingeki no Kyojin Gaiden: Kuinaki Sentaku', 'english': 'Attack on Titan: No Regrets', 'native': '進撃の巨人 外伝 悔いなき選択'}}, {'id': 20958, 'titles': {'romaji': 'Shingeki no Kyojin 2', 'english': 'Attack on Titan Season 2', 'native': '進撃の巨人２'}}]
Détails : {'id': 16498, 'title': {'romaji': 'Shingeki no Kyojin', 'english': 'Attack on Titan', 'native': '進撃の巨人'}, 'description': 'Several hundred years ago, humans were nearly exterminated by titans. Titans are typically several stories tall, seem to have no intelligence, devour human beings and, worst of all, s

### Modification pour récupérer les personnages et pas juste les animé

In [15]:
import requests
import json
import os

class AniListFetcher:
    """
    Classe pour interagir avec l'API AniList et récupérer des données sur les animes, mangas ou personnages.
    """

    API_URL = "https://graphql.anilist.co"

    def __init__(self):
        pass

    def search_titles(self, query, media_type="ANIME", max_results=5):
        """
        Recherche des titres sur AniList par type de média (ANIME, MANGA, CHARACTER).
        """
        if media_type == "CHARACTER":
            query_body = {
                "query": """
                    query ($search: String, $page: Int, $perPage: Int) {
                        Page(page: $page, perPage: $perPage) {
                            characters(search: $search) {
                                id
                                name {
                                    full
                                    native
                                }
                            }
                        }
                    }
                """,
                "variables": {
                    "search": query,
                    "page": 1,
                    "perPage": max_results
                }
            }
        else:
            query_body = {
                "query": """
                    query ($search: String, $type: MediaType, $page: Int, $perPage: Int) {
                        Page(page: $page, perPage: $perPage) {
                            media(search: $search, type: $type) {
                                id
                                title {
                                    romaji
                                    english
                                    native
                                }
                            }
                        }
                    }
                """,
                "variables": {
                    "search": query,
                    "type": media_type,
                    "page": 1,
                    "perPage": max_results
                }
            }

        response = requests.post(self.API_URL, json=query_body)
        data = response.json()

        if "errors" in data:
            raise ValueError(f"Erreur lors de la recherche : {data['errors']}")

        if media_type == "CHARACTER":
            results = [
                {
                    "id": character["id"],
                    "name": character["name"]
                }
                for character in data["data"]["Page"]["characters"]
            ]
        else:
            results = [
                {
                    "id": media["id"],
                    "titles": media["title"]
                }
                for media in data["data"]["Page"]["media"]
            ]

        if not results:
            raise ValueError("Aucun résultat trouvé pour cette recherche.")
        return results

    def fetch_details(self, media_id, media_type="ANIME"):
        """
        Récupère les détails d'un média (anime ou manga) par son ID.
        """
        query_body = {
            "query": """
                query ($id: Int, $type: MediaType) {
                    Media(id: $id, type: $type) {
                        id
                        title {
                            romaji
                            english
                            native
                        }
                        description
                        genres
                        episodes
                        chapters
                        volumes
                        coverImage {
                            large
                        }
                        averageScore
                        popularity
                    }
                }
            """,
            "variables": {
                "id": media_id,
                "type": media_type
            }
        }

        response = requests.post(self.API_URL, json=query_body)
        data = response.json()

        if "errors" in data:
            raise ValueError(f"Erreur lors de la récupération des détails : {data['errors']}")

        return data["data"]["Media"]

    def fetch_character_details(self, character_id):
        """
        Récupère les détails d'un personnage par son ID.
        """
        query_body = {
            "query": """
                query ($id: Int) {
                    Character(id: $id) {
                        id
                        name {
                            full
                            native
                        }
                        image {
                            large
                        }
                        description
                        media {
                            nodes {
                                title {
                                    romaji
                                    english
                                }
                                type
                            }
                        }
                    }
                }
            """,
            "variables": {
                "id": character_id
            }
        }

        response = requests.post(self.API_URL, json=query_body)
        data = response.json()

        if "errors" in data:
            raise ValueError(f"Erreur lors de la récupération des détails : {data['errors']}")

        return data["data"]["Character"]

    def save_to_json(self, data, path):
        """
        Enregistre les données dans un fichier JSON.
        """
        with open(path, 'w', encoding='utf-8') as file:
            json.dump(data, file, ensure_ascii=False, indent=4)
            print(f"Données enregistrées dans : {os.path.abspath(path)}")

    def save_to_html(self, data, path):
        """
        Enregistre les données dans un fichier HTML.
        """
        if "name" in data:  # Gestion des personnages
            html_content = f"""
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>{data['name']['full']}</title>
            </head>
            <body>
                <h1>{data['name']['full']} ({data['name']['native'] or 'N/A'})</h1>
                <img src="{data['image']['large']}" alt="{data['name']['full']}">
                <p><strong>Description :</strong> {data['description']}</p>
                <h2>Médias associés</h2>
                <ul>
            """
            for media in data["media"]["nodes"]:
                html_content += f"<li>{media['title']['romaji']} ({media['type']})</li>"

            html_content += """
                </ul>
            </body>
            </html>
            """
        else:  # Gestion des animes/mangas
            html_content = f"""
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>{data['title']['romaji']}</title>
            </head>
            <body>
                <h1>{data['title']['romaji']} ({data['title']['english'] or 'N/A'})</h1>
                <img src="{data['coverImage']['large']}" alt="{data['title']['romaji']}">
                <p><strong>Description :</strong> {data['description']}</p>
                <p><strong>Genres :</strong> {', '.join(data['genres'])}</p>
                <p><strong>Score moyen :</strong> {data['averageScore']}</p>
                <p><strong>Popularité :</strong> {data['popularity']}</p>
                <p><strong>Épisodes :</strong> {data.get('episodes', 'N/A')}</p>
                <p><strong>Chapitres :</strong> {data.get('chapters', 'N/A')}</p>
                <p><strong>Volumes :</strong> {data.get('volumes', 'N/A')}</p>
            </body>
            </html>
            """

        with open(path, 'w', encoding='utf-8') as file:
            file.write(html_content)
            print(f"Données enregistrées en HTML dans : {os.path.abspath(path)}")


# Exemple d'utilisation
if __name__ == "__main__":
    fetcher = AniListFetcher()

    # Recherche d'un personnage
    search_results = fetcher.search_titles("Levi", media_type="CHARACTER")
    print("Résultats de recherche :", search_results)

    # Récupération des détails du personnage
    character_id = search_results[0]["id"]
    details = fetcher.fetch_character_details(character_id)
    print("Détails :", details)

    # Sauvegarde des données
    fetcher.save_to_json(details, "character_data.json")
    fetcher.save_to_html(details, "character_data.html")


Résultats de recherche : [{'id': 45627, 'name': {'full': 'Levi', 'native': 'リヴァイ'}}, {'id': 52885, 'name': {'full': 'Levi Kazama', 'native': '風間レヴィ'}}, {'id': 216491, 'name': {'full': 'Leviathan', 'native': 'レヴィアタン'}}, {'id': 41838, 'name': {'full': 'Shion Eliphas Levi', 'native': 'シオン エリファス レビィ '}}, {'id': 183509, 'name': {'full': 'Serafall Leviathan', 'native': 'セラフォルー'}}]
Détails : {'id': 45627, 'name': {'full': 'Levi', 'native': 'リヴァイ'}, 'image': {'large': 'https://s4.anilist.co/file/anilistcdn/character/large/b45627-CR68RyZmddGG.png'}, 'description': '__Height:__ 160 cm (5\'3")\n\nSquad Captain of the Special Operations Squad (known as Squad Levi) within the Survey Corps. He is known as humanity\'s strongest soldier. \n\nLevi is abrasive, blunt, and has a strong respect for structure and discipline. He is a clean-freak, demanding spotless cleanliness of himself, his equipment, and his environment. Rumors say he was involved in underground crime in his life before coming a soldier.